In [ ]:
!pip install kagglehub numpy matplotlib pandas

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivamb/bank-customer-segmentation")

print("Path to dataset files:", path)

Path to dataset files: /home/margo/.cache/kagglehub/datasets/shivamb/bank-customer-segmentation/versions/1


In [9]:
raw_data_path = "../data/initial_dataset/raw"

# Создаем папку назначения
!mkdir -p {raw_data_path}

# Копируем данные
!cp -r {path}/* {raw_data_path}/

# Проверяем
print("Скопированные файлы:")
!ls {raw_data_path}/

Скопированные файлы:
bank_transactions.csv


In [10]:
import pandas as pd
df = pd.read_csv(raw_data_path + "/bank_transactions.csv")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [12]:
df.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
count,1.046198e+06,1.048567e+06,1.048567e+06
mean,1.154035e+05,1.570875e+05,1.574335e+03
std,8.464854e+05,5.126185e+04,6.574743e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.721760e+03,1.240300e+05,1.610000e+02
50%,1.679218e+04,1.642260e+05,4.590300e+02
75%,5.765736e+04,2.000100e+05,1.200000e+03
max,1.150355e+08,2.359590e+05,1.560035e+06


Обнаруженные проблемы:
1) Пропуски в CustomerDOB, CustLocation, CustAccountBalance. Действие: визуализировать и посмотреть, небольшой процент пропусков, поэтому возможно можно заполнить модой или создать категорию "неизвестно"
2) TransactionTime в неправильном формате. Действие: Нужно преобразовать в строку и разбить на часы/минуты/секунды
3) CustAccountBalance как будто есть выбросы. Действие: применить логарифмирование или масштабирование при создании признаков. Пропуски заполнить медианой или создать флаг is_balance_missing.
4) TransactionAmount есть нулевая сумма транзакции, что странно, и есть слишком большая сумма транзакций. Действие: нужно лучше понять построив график. Нулевые суммы возможно, удалить, так как для прогноза суммы они неинформативны. Возможно использовать логарифмирование или отсечь 99-й перцентиль, чтобы модель не смещалась из за больших значений.
5) TransactionDate как object. Действие: Нужно преобразовать в datetime.